***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/brain_stroke.csv')
df = df.dropna()

encoder = OrdinalEncoder()
df['gender'] = encoder.fit_transform(df[['gender']])
df['ever_married'] = encoder.fit_transform(df[['ever_married']])
df['work_type'] = encoder.fit_transform(df[['work_type']])
df['Residence_type'] = encoder.fit_transform(df[['Residence_type']])
df['smoking_status'] = encoder.fit_transform(df[['smoking_status']])

X = df.loc[:, df.columns != 'stroke']
y = df['stroke']

rus = RandomOverSampler(random_state=42)
X, y = rus.fit_resample(X, y)
print(len(X), len(y))

9466 9466


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)
cv = StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

In [ ]:
best = LinearSVC(C=5, random_state=42)
best.fit(X, y)

cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')

print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

train_predictions = best.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.33309859 0.35334681 0.70919042 0.49903818 0.76665472 0.70384818
 0.34034207 0.66969274 0.75558426 0.33333333]
Mean CV Accuracy: 0.5464129311655487
Training Set Accuracy: 0.6546587787872385


##**Extracting weights and intercept values**

In [ ]:
b = best.intercept_
w = best.coef_

w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['stroke'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'Stroke_Predicted': predicted_label})
index_class = results_df.index[results_df['Stroke_Predicted'] == 1]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

3355


##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

##**Optimizer -- step 1**

In [ ]:
from z3 import *
import csv
import time

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    # Gender
    original_value = og_x_np[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    # Age
    original_value1 = og_x_np[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    # hypertension
    opt.add(Or(X[2] == 0, X[2] == 1))

    # heart disease
    opt.add(Or(X[3] == 0, X[3] == 1))

    # married or not
    original_value4 = og_x_np[4]
    if original_value4 is not None:
        opt.add(X[4] == original_value4)

    # worktype
    worktype = Or(X[5] == 0, X[5] == 1, X[5] == 2, X[5] == 3, X[5] == 4)
    opt.add(worktype)

    # residence
    opt.add(Or(X[6] == 0, X[6] == 1))

    # glucose level
    constraint6 = And(X[7] >= 70, X[7] <= 127)
    opt.add(constraint6)

    # bmi
    constraint7 = And(X[8] >= 18.5, X[8] <= 25)
    opt.add(constraint7)

    #smoking status
    smoking = Or(X[9] == 0, X[9] == 1, X[9] == 2, X[9] == 3)
    opt.add(smoking)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B < 0
    opt.add(inequality)

    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(l1_distance).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step1.csv', names = column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 2**

In [ ]:
def step2(W, B, row):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    # Gender
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    # Age
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    # hypertension
    opt.add(Or(X[2] == 0, X[2] == 1))

    # heart disease
    opt.add(Or(X[3] == 0, X[3] == 1))

    # married or not
    original_value4 = row[4]
    if original_value4 is not None:
        opt.add(X[4] == original_value4)

    # worktype
    worktype = Or(X[5] == 0, X[5] == 1, X[5] == 2, X[5] == 3, X[5] == 4)
    opt.add(worktype)

    # residence
    opt.add(Or(X[6] == 0, X[6] == 1))

    #glucose level
    slack_glucose = Real('slack_glucose')
    slack_glucose_constr = (slack_glucose >= 0)
    constraint6 = And(X[7] > 0, X[7] >= 70 - slack_glucose, X[7] <= 127 + slack_glucose)
    opt.add(constraint6, slack_glucose_constr)

    #bmi
    slack_bmi = Real('slack_bmi')
    slack_bmi_constr = (slack_bmi >= 0)
    constraint7 = And(X[8] > 0, X[8] >= 18.5 - slack_bmi, X[8] <= 25 + slack_bmi)
    opt.add(constraint7, slack_bmi_constr)

    #smoking status
    smoking = Or(X[9] == 0, X[9] == 1, X[9] == 2, X[9] == 3)
    opt.add(smoking)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])
    inequality = dot_product + B < 0
    opt.add(inequality)

    objective =  (slack_glucose + slack_bmi)
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        slack_glucose = model[slack_glucose].as_decimal(15)
        print(f"slack_glucose value: {slack_glucose}")

        slack_bmi = model[slack_bmi].as_decimal(15)
        print(f"slack_bmi value: {slack_bmi}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_glucose, slack_bmi

    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '']

W = w
B = b[0]
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()
    original_f, x_prime_optimized, cost, cost_optimized, slack_glucose, slack_bmi = step2(W, B, row)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_glucose] + [slack_bmi]
        writer.writerow(data_row)

Streaming output truncated to the last 5000 lines.
Optimized x': ['1', '74', '0', '0', '1', '0', '0', '36.190421275502589?', '18.5', '3']
Optimized distance: 33.809578724497410?
Optimized cost: 33.809578724497410?
slack_glucose value: 33.809578724497410?
slack_bmi value: 0
Optimized x': ['0', '59', '0', '0', '1', '0', '0', '57.255415304806373?', '18.5', '3']
Optimized distance: 12.744584695193626?
Optimized cost: 12.744584695193626?
slack_glucose value: 12.744584695193626?
slack_bmi value: 0
Optimized x': ['1', '75', '0', '0', '1', '0', '0', '34.470363981252888?', '18.5', '3']
Optimized distance: 35.529636018747111?
Optimized cost: 35.529636018747111?
slack_glucose value: 35.529636018747111?
slack_bmi value: 0
Optimized x': ['0', '77', '0', '0', '1', '0', '0', '26.294384008311760?', '18.5', '3']
Optimized distance: 43.705615991688239?
Optimized cost: 43.705615991688239?
slack_glucose value: 43.705615991688239?
slack_bmi value: 0
Optimized x': ['1', '76', '0', '0', '1', '0', '0', '32.75

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_glucose', 'Slack_bmi']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 3**

In [ ]:
def step3(W, B, row, slack_value1, slack_value2):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Real(f'x_{i}') for i in range(n)]

    # Gender
    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    # Age
    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    # hypertension
    opt.add(Or(X[2] == 0, X[2] == 1))

    # heart disease
    opt.add(Or(X[3] == 0, X[3] == 1))

    # married or not
    original_value4 = row[4]
    if original_value4 is not None:
        opt.add(X[4] == original_value4)

    # worktype
    worktype = Or(X[5] == 0, X[5] == 1, X[5] == 2, X[5] == 3, X[5] == 4)
    opt.add(worktype)

    # residence
    opt.add(Or(X[6] == 0, X[6] == 1))

    #glucose level
    constraint6 = And(X[7] >= 70 - slack_value1, X[7] <= 127 + slack_value1)
    opt.add(constraint6)

    #bmi
    constraint7 = And(X[8] >= 18.5 - slack_value2, X[8] <= 25 + slack_value2)
    opt.add(constraint7)

    #smoking status
    smoking = Or(X[9] == 0, X[9] == 1, X[9] == 2, X[9] == 3)
    opt.add(smoking)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    # Create the inequality constraint W * X + B >= 0
    inequality = dot_product + B < 0
    opt.add(inequality)

    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective)
        print(f"Distance: {l1_distance_1}")
        cost_optimized = model.eval(objective)
        print(f"Objective: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")

        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l0/Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_glucose'].iloc[i]
    slack_value2 = last['Slack_bmi'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2)

    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l0/Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l0/Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0 and l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l0/Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step1_optimized.csv')
d = pd.concat([d1, d3_initial])

l1_runtime = d.loc[:, 'Runtime'].mean()
print("L1 runtime", l1_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

43.01043219076006


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/BrainStroke_l1/Step2.csv')
number_of_slacks_per_record = 2

count_s1 = 0
count_s2 = 0
total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_glucose'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['Slack_glucose'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_bmi'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_bmi'].iloc[i] == 0:
        count_s2 = 0

    total_count = count_s1 + count_s2
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)

50.0
